In [24]:
import pandas as pd
import numpy as np
from itertools import combinations
from poker_utils.constants import HANDS_DICT
import random

In [ ]:
# ev_data = pd.read_csv("data/raw/ev_pca_data.csv")

In [21]:
random.shuffle(hands)

In [28]:
edge_list

[('Q2o', 'T4o'), ('Q2o', 'A5o'), ('Q2o', 'J4s'), ('Q2o', 'J6o')]

In [ ]:

edge_list = []
for hand1, hand2 in combinations(HANDS_DICT.values(), 2):
    answer = int(input(f"Are these cards similar (1,0) {hand1} and {hand2}: "))
    if answer == 1:
        edge_list.append((hand1,hand2))
        

KeyboardInterrupt: Interrupted by user

In [3]:
def process_base_hand_data(path_to_data: str) -> pd.DataFrame:
    raw_hand_data = pd.read_csv(path_to_data)
    raw_hand_data = raw_hand_data.rename(columns={
        'equity.flop': 'flop_equity',
        'equity.turn': 'turn_equity',
        'equity.river': 'river_equity'})
    raw_hand_data = raw_hand_data.assign(
        equity_delta=raw_hand_data['river_equity']-raw_hand_data['flop_equity']
        )
    raw_hand_data.set_index("hand_idx", inplace=True)
    raw_hand_data.index.name = "hand"
    return raw_hand_data[
        ['connectedness','suited','pair','flop_equity','turn_equity','river_equity','equity_delta']
        ]

    

In [ ]:
process_base_hand_data("data/raw/hand_")

In [16]:
def process_dom_block_data(path_to_data: str) -> pd.DataFrame:
    raw_data = pd.read_csv(path_to_data)
    raw_data['hand'] = raw_data['hand'].apply(lambda x: {j:i for i,j in HANDS_DICT.items()}[x])
    raw_data.set_index('hand', inplace=True)
    
    return raw_data[
    ['combos_dominated', 'dom_ratio', 'block_ratio_cat', 'dom_block_ratio', 'cluster']
    ]
    
    

In [ ]:
# def process_hand_ev_data(path_to_data: str) -> pd.DataFrame:
#     raw_data = pd.read_excel(path_to_data)
#     raw_data['hand'] = raw_data['hand'].apply(lambda x: {j:i for i,j in HANDS_DICT.items()}[x])
    